In [1]:
import pandas as pd
import json

In [2]:
folder="base_data/"

In [3]:
ment=pd.read_csv(folder+"sodhganga_mentorship_dept_rev.csv")

In [4]:
ment.loc[ment["uniq_ddc_across_dept"]=="('',)",'DepartmentId']=ment[ment["uniq_ddc_across_dept"]=="('',)"]['N_DepartmentId_1']

In [5]:
ment.columns

Index(['advisorId', 'researcherId', 'advisor_name', 'researcher_name',
       'date_submitted', 'dc.language.iso[]', 'publisher_dept',
       'N_DepartmentId_1', 'publisher_institution', 'instituteId', 'title',
       'thesisId', 'dc.type.degree', 'dc.date.awarded', 'dc.subject.ddc',
       'dc.description.abstract', 'dc.title.alternative', 'N_DepartmentId',
       'uniq_ddc_across_dept', 'DepartmentId'],
      dtype='object')

In [6]:
ment.shape

(205220, 20)

In [7]:
ment['previous_advisor_name']=ment['advisor_name']
ment['date_submitted']=pd.to_datetime(ment['date_submitted'],errors='coerce')
ment['dc.date.awarded']=pd.to_datetime(ment['dc.date.awarded'],errors='coerce')

In [8]:
ment.loc[ment['advisor_name'].str.count(",")>1,'advisor_name']=ment[ment['advisor_name'].str.count(",")>1]['advisor_name'].str.replace(r"[^\u0000-\u05C0\u2100-\u214F]+","")

In [9]:
ment.loc[ment['advisor_name'].str.contains("\(.*\)"),'advisor_name']=ment[ment['advisor_name'].str.contains("\(.*\)")]['advisor_name'].str.replace(r"\(.*\)","")

In [10]:
#ment[ment['advisor_name'].str.contains('Ibraheem')]

In [11]:
#ment.loc[{1366, 1390, 129114, 129115, 129133, 129396, 129436, 184906}]['advisor_name']

In [12]:
name_wit_and=["Andrea","Andrew","Andotra",'Andre',"Andal"]

In [13]:
name_correction_dict = {'Asif Husain, Shahar Yar, Mohammad':'Asif Husain; Shahar Yar, Mohammad',
'Ahmad, Farhan Jalees, Zeenat Iqbal':'Ahmad, Farhan Jalees; Zeenat Iqbal',
'Aldous, V. Huxley, J.':'V. Aldous J. Huxley',
'Ali, Saaduz, Zafar':"Saaduz Zafar Ali",
'Aravindakumar, C. T% Usha, K.':'Aravindakumar, C. T; Usha, K.',
'Arunachalam, Nand Ramanujam, N.':'Arunachalam, N; Ramanujam, N.',
'Asha Sharma, Gaurav Sharma, Pushpa Mehta':'Asha Sharma; Gaurav Sharma; Pushpa Mehta',
'Bais, V. S. Nad Soni, N. K.':'Bais, V. S. ; Soni, N. K.',
'Biju, P. R% Unnikrishnan, N. V.':'Biju, P. R; Unnikrishnan, N. V.',
'Chacko, Km , Sarwat Sultana':'Chacko, Km ; Sarwat Sultana',
'Chandrakumar, P. S. Amd Mohan, S.':'Chandrakumar, P. S.; Mohan, S.',
'Chatterjee, N. C, Dutta Sikha':'Chatterjee, N. C; Dutta Sikha',
'Chhabra, Ravi Kand Banerjee, B. D.':'Chhabra, Ravi K ; Banerjee, B. D.',
'Dar, Abid Hamid, Shakir Ali':'Dar, Abid Hamid; Shakir Ali',
'Dubey, Chandra Dattnd Singh, Gajendra Pal':'Dubey, Chandra Datt ; Singh, Gajendra Pal',
'Gayatri Rath, Sunita Saxena, Ranju Ralhan':'Gayatri Rath; Sunita Saxena; Ranju Ralhan',
'Hadrikar, Kalyani, Anand':'Hadrikar, Kalyani Anand',
'James, F. Daniel, R.':'James, F. ; Daniel, R.',
'Joseph, Cyriac% Unnikrishnan, N. V.':'Joseph, Cyriac; Unnikrishnan, N. V.',
'Jyothi P, Gulati, P.':'Jyothi P; Gulati, P.',
'Kalarikkal, Nandakumar% Thomas, Sabu':'Kalarikkal, Nandakumar; Thomas, Sabu',
'Kamili, Azra N, Ruqeya Nazir':'Kamili, Azra N; Ruqeya Nazir',
'Kavita Khanna; Akansha Singh; Gaurav Gupta':'Kavita Khanna, Akansha Singh, Gaurav Gupta',
'Kjmahale, Mtarinayya, Ir Babu Singh':'Kjmahale; Mtarinayya Ir; Babu Singh',
'Kohli, Kanchan, Mushir Ali':'Kohli, Kanchan; Mushir Ali',
'Kumaria, Suman Tandon, Pramod':'Kumaria, Suman; Tandon, Pramod',
'Kurian, Thomas Alex, Rosamma':'Kurian, Thomas; Alex, Rosamma',
'Lohani, Mohtashim, Qamar Rehman':'Lohani, Mohtashim; Qamar Rehman',
'Maan, A. Sand Goyal, D. R.':'Maan, A. S ; Goyal, D. R.',
'Maheshwari, H. Sand Jain, Subodhkumar':'Maheshwari, H. S ; Jain, Subodhkumar',
'Meyyanathan, S, N.':'Meyyanathan, S. N.',
'Najmi, Abul Kalam, Sayeed Ahmad':'Najmi, Abul Kalam; Sayeed Ahmad',
'Pal, Sc , Ritu Gilthotra':'Pal, Sc ; Ritu Gilthotra',
'Prasad, Gbks Co-gudie Vijayaraghavan, R.':'Prasad, Gbks ; Vijayaraghavan, R.',
'Pujari, A. K. Kannan V. Ashutosh Gulati, V. P.':'Pujari, A. K.; Kannan V.; Saxena, Ashutosh; Gulati, V. P.',
'Pujari, A. K. Saxena, Ashutosh Gulati, V. P.':'Pujari, A. K.; Saxena, Ashutosh; Gulati, V. P.',
'Ramachandra Panikkar, K% Mukkadan, J. K.':'Ramachandra Panikkar, K; Mukkadan, J. K.',
'Ramakrishna Rao, Band Madhusudhana Rao, M.':'Ramakrishna Rao, B ; Madhusudhana Rao, M.',
'Rao, Rama S. Tandon, Pramod':'Rao, Rama S.; Tandon, Pramod',
'Rk Manohar, Sudhinder Singh Chowhan, Ibrahim Yahaya':'Rk Manohar; Sudhinder Singh Chowhan; Ibrahim Yahaya',
'Sadaf Siraj, Ravichandran, N.':'Sadaf Siraj; Ravichandran, N.',
'Sayeed Ahmad, Husain, Syed Akhtar':'Sayeed Ahmad; Husain, Syed Akhtar',
'Shakir Ali, Tandon, Monika':'Shakir Ali; Tandon, Monika',
'Sheeba Fareed, Siddiqui, Hh':'Sheeba Fareed; Siddiqui, Hh',
'Sheeba Farid, Siddiqui, Hh':'Sheeba Fareed; Siddiqui, Hh',
'Shivakumar, S, Chengti':'Shivakumar; S, Chengti',
'Shobha Das, Mohammad Fahim, Jagdish Prasad':'Shobha Das; Mohammad Fahim; Jagdish Prasad',
'Siddiqui, Waseem Ahmad, Mohd Mujeeb':'Siddiqui, Waseem Ahmad; Mohd Mujeeb',
'Singh, Surendra Pal , Dharminder Kumar':'Singh, Surendra Pal ; Dharminder Kumar',
'Syad, Asif Ali Haleem, Abid':'Syad, Asif Ali; Haleem, Abid',
'Thomas, Sabu% Kalarikkal, Nandakumar':'Thomas, Sabu; Kalarikkal, Nandakumar',
'Thomas, Sabu% Mathew, Suresh':'Thomas, Sabu; Mathew, Suresh',
'Tiwari, Vivekanand Pandey, Ganga Prasad':'Tiwari, Vivekanand ;Pandey, Ganga Prasad',
'Tomar, Rajesh Singh Prasad, G. B. K. S.':'Tomar, Rajesh Singh; Prasad, G. B. K. S.',
'Vaishnavi, Samanthaand Singh, Shashank K.':'Vaishnavi, Samantha; Singh, Shashank K.',
'Yogesh Chandra Sharma, K. J. Rangra, Vaibhav Neema':'Yogesh Chandra Sharma; K. J. Rangra; Vaibhav Neema',
'Zafar, Sm Tariq Gupta, Parul':'Zafar, Sm Tariq; Gupta, Parul',
'Rajeev Kumar , K. , Jayaraj Mk':'Rajeev Kumar , K. ; Jayaraj Mk',
'Azmi, Feza Tavassaum Kumar, Ravindra':'Azmi, Feza Tavassaum ; Kumar, Ravindra',
'Nitin Malik, A. N. Jha, B. K. Panigrahi':'Nitin Malik ; A. N. Jha ; B. K. Panigrahi',
'Kavita Khanna, Akansha Singh, Gaurav Gupta':'Kavita Khanna; Akansha Singh; Gaurav Gupta',
'Janbu, M. Pharm, Phd': 'Janbu, M.',
'Tomar, R. S. Singh, R. K.': 'Tomar, R. S.; Singh, R. K.',
'Kumud Upadhyaya, Kumau University, Bhimtal Nanital Uknv Satheesh Madhav, Director, Faculty Of Pharmacy, Dituniversity Dehradun, Uk':'Kumud Upadhyaya; Satheesh Madhav',
'Agarwal, P, Rajendra, Tipathi, S, P, 06':'Agarwal, P. Rajendra; Tipathi, S. P. ',
'Vankar, R. H. Sneh Swarup':'Vankar, R. H. ; Sneh Swarup',
'Ramesh V. , Umashankars':'Ramesh V. ; Umashankar S.',
'Mathew, Linu Augustine Jomy':'Mathew, Linu; Augustine Jomy',
'Mathew, Linu Padmakumar K. G.':'Mathew, Linu; Padmakumar K. G.',
'Gayatri Rath, Sunita Saxena':'Gayatri Rath; Sunita Saxena',
'Arora, Sunita, Ratna':'Arora, Sunita; Ratna',
'Nagaraj Bn, Prasad Vs':'Nagaraj Bn; Prasad Vs',
'Vjzacharias, Sabu K. Thomas':'Vjzacharias; Sabu K. Thomas',
'Vjzacharias, Tm Benny':'Vjzacharias; Tm Benny',
'Mathew, Linu Augustine Jomy':'Mathew, Linu; Augustine Jomy',
'Mathew, Linu Padmakumar K. G.':'Mathew, Linu ; Padmakumar K. G.',
'Alimuddin, Ravi Kumar':'Alimuddin; Ravi Kumar',
"Sony Kumari, Akhilesh K. B, Nagendra H. R.":'Sony Kumari; Akhilesh K. B; Nagendra H. R.',
'Epurushothaman, Gunnikrishnan':'Epurushothaman; Gunnikrishnan',
'Cjanardanan, Kkaravindakshan':'Cjanardanan; Kkaravindakshan',
'Srivastava Deepak Chugan P. K.':'Srivastava Deepak; Chugan P. K.',
'Lourthusamy S. S. Epanesar, S.':'Lourthusamy S. S.; Epanesar, S.',
'Jangra, Gulabchand K. Gupta Surender':'Gulabchand K. Gupta; Surender Jangra',
'C. Renuka, Msivadasan':'C. Renuka; Msivadasan',
'Sony Kumari, Akhilesh K. B, Nagendra H. R.':'Sony Kumari; Akhilesh K. B; Nagendra H. R.',
'Panikkar, Radhakrishna Pillai M. Ramachandra':'Radhakrishna Pillai M.; Ramachandra Panikkar',
'Karanth Prathibha, Rajashekar B.':'Karanth Prathibha; Rajashekar B.',
'Y. Narasimha Reddy, Gv Subba Reddy':'Y. Narasimha Reddy; Gv Subba Reddy',
'Biplab De, N. Devanna':'Biplab De; N. Devanna',
'Vikas Hooda, Priyanka Siwach':'Vikas Hooda; Priyanka Siwach',
'Mohan, R. Manivel, M.':'Mohan, R.; Manivel, M.',
'Mewada Raju, Devashrayee N. M.':'Mewada Raju; Devashrayee N. M.',
'Sayeed Ahmad, Mohd Mujeeb':'Sayeed Ahmad; Mohd Mujeeb',
'Gupta, Krishna Prabha , Sarwat Sultana':'Gupta, Krishna Prabha ; Sarwat Sultana',
'Gayatri Rath, Ranju Ralhan':'Gayatri Rath; Ranju Ralhan',
'Gaurav Sharma, Rajesh Kumar Yadav':'Gaurav Sharma; Rajesh Kumar Yadav',
'Ritu M. Gilhotra, Dhanajay B. Meshram':'Ritu M. Gilhotra; Dhanajay B. Meshram',
'Kr Dayas, Kkrishnankutty':'Kr Dayas; Kkrishnankutty',
'C. Renuka, Aleyamma Thomas':'C. Renuka; Aleyamma Thomas',
'Das M. R. Karunagaran D.':'Das M. R.; Karunagaran D.',
'Bhoraskar, Dhole Sanjay D.':'V.N. Bhoraskar; Dhole Sanjay D.',
'Singh Lokendra, Kalita Mc':'Singh Lokendra; Kalita Mc',
'Rajkhowa Pranjan, Saikia Helen K.':'Rajkhowa Pranjan; Saikia Helen K.',
"Mohan Girish, K. S.":"Mohan; Girish, K. S.",
"Gireesha B. J. , Bagewadi C. S.":"Gireesha B. J. ; Bagewadi C. S.",
"Narasimhamurthy S. K. Bagewadi C. S.":"Narasimhamurthy S. K.; Bagewadi C. S.",
"Venkatesha , Bagewadi C. S.":"Venkatesha ; Bagewadi C. S.",
"Snmamle Desai, Girendra Gautam":"Snmamle Desai; Girendra Gautam",
"Jalalpure Ss , Girendra Gautam":"Jalalpure Ss ; Girendra Gautam",
'B. Arul, Girendra Gautam':'B. Arul; Girendra Gautam',
"Anurag, Girendra Gautam":'Anurag; Girendra Gautam',
'Mandev B. Patel, Girendra Gautam':'Mandev B. Patel; Girendra Gautam',
'Jalalpure Ss , Girendra Gautam':'Jalalpure Ss ; Girendra Gautam',
'B. Arul, Girendra Gautam':'B. Arul; Girendra Gautam',
'Gaurav Sharma , Nakuleshwar Dut Jasuja':'Gaurav Sharma ; Nakuleshwar Dut Jasuja',
'Gaurav Sharma, Nakuleshwar Dut Jasuja':'Gaurav Sharma; Nakuleshwar Dut Jasuja',
'Ravindra Pal Singh, Gn Darwhekar':'Ravindra Pal Singh; Gn Darwhekar',
'Ravindra Pal Singh, Mousumi Kar':'Ravindra Pal Singh; Mousumi Kar',
'Ravindra Pal Singh, G. Vidyasagar':'Ravindra Pal Singh; G. Vidyasagar',
'Krishna Chandra Roy, Dinesh Goyal':'Krishna Chandra Roy; Dinesh Goyal',
'Sanjay Singla , Dinesh Goyal':'Sanjay Singla ; Dinesh Goyal',
'Baldev Singh, Ripu Ranjan Sinha':'Baldev Singh; Ripu Ranjan Sinha',
'Ripu Ranjan Sinha, Pc Gupta':'Ripu Ranjan Sinha; Pc Gupta',
'Srinivasan T. M, Alex Hankey':'Srinivasan T. M; Alex Hankey',
'Sony Kumari, Akhilesh K. B.':'Sony Kumari; Akhilesh K. B.',
'V. Rama Mohan Gupta, N. Devanna':'V. Rama Mohan Gupta; N. Devanna',
'Y. Padmanabha Reddy, N. Devanna':'Y. Padmanabha Reddy; N. Devanna',
'V. Rama Mohan Gupta, N. Devanna':'V. Rama Mohan Gupta; N. Devanna',
'Kand Rath, P. C.':'Bamezai, R N K ; Rath, P C'
}

In [14]:
len(name_correction_dict)

123

In [15]:
with open(folder+'name_mapping_manually.txt', 'w') as outfile:
    json.dump(name_correction_dict, outfile)

In [16]:
ment.replace({"advisor_name": name_correction_dict},inplace=True)

In [17]:
name_with_correct_and=['Randhawa Gurinderjit','Sahastrabudhe','Shrivastava Rk','Sharma, Bk',"Bhadauria Rekha",'Prasad Gbks',
              'Srivastava Nalini','kamala Devi','Rc Chhipa','Dhakad, Rk','Goswami, Dn']

In [18]:
remove=[",Mcom, Mba, Mphil, Phd,",
"Associate , Doon University, Dehradun",
'Mcom, Mba, Mphil',
'Head, Department Of Applied Sciences, Gl Bajaj Institute Of Technology',
"Former Scientist, Defence Electronics Application Lab, Drdo, Dehradun",
"Usic, Hnbgu, Srinagar-garhwal",
"Dit Dehradun Pattnaik Ss, Nitttr , Chandigarh",
"Kumau University, Bhimtal Nanital Uknv",
"Director, Faculty Of Pharmacy, Dituniversity Dehradun, Uk",
"Director Institute Of Technology, Gopeshwar, Uttarakhand"
"Bt Kumaon Institute Of Technology, Dwarahat",
"Mechanical Engineering Department Hbtu, Kanpur",
"Mechanical Engineering Department Dtu, New Delhi",
"Phd,",
"Department",
"Mpharm,* Phd,*",
"Director Institute Of Technology, Gopeshwar, Uttarakhand",
"Msc, M. Phil, Ph D",
"À¤¡à¤¿à¤¸à¥à¤à¤¾, À¤à¤à¤¦à¥à¤°à¤²à¥à¤à¤¾ À¤à¤° À¤­à¥à¤à¤¬à¥à¤°à¥ , À¤à¤¦à¤¯",
"À¤¡à¤¿à¤¸à¥à¤à¤¾; À¤à¤à¤¦à¥à¤°à¤²à¥à¤à¤¾ À¤à¤° À¤­à¥à¤à¤¬à¥à¤°à¥ , À¤à¤¦à¤¯",
"À¤¸à¤°à¤¦à¥à¤¸à¤¾à¤¯, À¤®à¤¾à¤§à¤µà¥ À¤à¤° À¤­à¥à¤à¤¬à¥à¤°à¥, À¤à¤¦à¤¯",
"À¤¸à¤°à¤¦à¥à¤¸à¤¾à¤¯; À¤®à¤¾à¤§à¤µà¥ À¤à¤° À¤­à¥à¤à¤¬à¥à¤°à¥, À¤à¤¦à¤¯",
"Àª®àª¿àª¸à«\x8dàª¤à«\x8dàª°à«\x80, Àª®àª¿àª²àª¨ Àª\x9fà«\x80",
"À¤¡à¤¿à¤¸à¥\x8cà¤\x9dà¤¾, À¤\x9aà¤¾à¤\x82à¤¦à¥\x8dà¤°à¤²à¥\x87à¤\x96à¤¾",
"À¤¡à¤¿à¤¸à¥\x8cà¤\x9dà¤¾, À¤\x9aà¤\x82à¤¦à¥\x8dà¤°à¤²à¥\x87à¤\x96à¤¾",
"Àª¸àª°àª¦àª¾àª°, Àª\x8fàª\x9aàª¸à«\x80",
"À¤\x9aà¤¤à¥\x81à¤°à¥\x8dà¤µà¥\x87à¤¦à¥\x80 À¤¶à¥\x81à¤\x95à¤¦à¥\x87à¤µ",
'à¤°à¤¾à¤\x9cà¤\x95à¥\x81à¤®à¤¾à¤°à¤¦à¥\x8dà¤°à¤¿à¤µà¥\x87à¤¦à¤¿',
"Bt Kumaon Institute Of Technology, Dwarahat",
"Â\x80\x98rohitshvaâ\x80\x99",
"à¤µà¤¿à¤¶à¤¨à¤²à¤¾à¤² À¤\x97à¥\x8cà¤¡à¤¼ À¤µà¥\x8dà¤¯à¥\x8bà¤®à¥\x8dà¤¶à¥\x87à¤\x96à¤°",
"Aà®£à¯\x8dà®\x9fà¯\x81à®°à®\x99à¯\x8dà®\x95à®©à¯\x8d, À®",
"Aà®ªà®¾à®£à¯\x8dà®\x9fà¯\x81à®°à®\x99à¯\x8dà®\x95à®©à¯\x8d, À®",
"Eee"
]

In [19]:
with open(folder+'remove_from_names.txt', 'w') as outfile:
    json.dump(remove, outfile)

In [20]:
spliter=['Co-guideand',"Guide:?","Co-*supervisor:?", "/", "Co-*supv-","Co-*guide","Co\s*Supervisor","Co\s*-?guide","Co\s*Guide","Co-\s*Guide","Co-*Supervisor"]

In [21]:
regex_pat1="|".join(spliter)

In [22]:
regex_pat='|'.join(remove)

In [23]:
ment.shape

(205220, 21)

In [24]:
#index1=ment[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False)]['advisor_name'].index

In [25]:
#set(index1)-set(index2)

In [26]:
ment['advisor_name']=ment['advisor_name'].str.replace(regex_pat,"",regex=True)

In [27]:
ment['advisor_name']=ment['advisor_name'].str.replace(regex_pat1," And ",regex=True)

In [28]:
ment[ment["previous_advisor_name"].str.contains("Ali Hasan Co Supervisor Khan, Ra")][['previous_advisor_name',"advisor_name"]]

,previous_advisor_name,advisor_name
81795,"Ali Hasan Co Supervisor Khan, Ra","Ali Hasan And Khan, Ra"


In [29]:
ment.loc[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False),'advisor_name']=ment[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False)]['advisor_name'].str.strip().str.replace(r'^And\s',"")
ment.loc[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False),'advisor_name']=ment[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False)]['advisor_name'].str.strip(' :')

In [30]:
#ment[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False)]['advisor_name']

In [31]:
ment.shape

(205220, 21)

In [32]:
def preprocessing(name):
    tmp=[a in name for a in name_wit_and]
    if any(tmp):
        return "not_include"
    elif any([a in name for a in name_with_correct_and]):
        flag=True
    else:
        name = name.split("And")
        flag=True
        if "," in name[-1]:
            flag=True
        elif " " in name[0]:
            comma_part=name[0].split(" ")
            if "," in comma_part[0]:
                flag=False
        else:
            flag=True
    return flag

In [33]:
#ment.loc[{1366, 1390, 129114, 129115, 129133, 129396, 129436, 184906}]['advisor_name']

In [34]:
ment['and_order_correct']=ment[ment['advisor_name'].str.contains(r'\sAnd\s?',case=False)]['advisor_name'].apply(lambda x : preprocessing(x))

In [35]:
#ment[ment['advisor_name'].str.contains(r'\sAnd[a-zA-z]+',case=False)]['advisor_name']

In [36]:
#preprocessing('Patel, Rakeshkumar Jat And Vishnu Motilal')

In [37]:
#ment[ment['and_order_correct']==False]['advisor_name'].values

In [38]:
tmp2=pd.DataFrame()

In [39]:
tmp2[['last','first']]=ment[ment['and_order_correct']==False]['advisor_name'].str.split(',',n=1,expand=True)

In [40]:
tmp2[['last','first']]=tmp2[['last','first']].fillna("")

In [41]:
ment.loc[ment['and_order_correct']==False,'advisor_name']=tmp2['first']+" "+tmp2['last']

In [42]:
#ment.loc[~(ment['and_order_correct'].isnull()),'advisor_name']=ment[~(ment['and_order_correct'].isnull())]['advisor_name'].str.strip()

In [43]:
#ment[ment['and_order_correct']==True]['advisor_name'].values

In [44]:
#ment[ment['advisor_name'].str.contains(r'\sAnd\s?') & (ment['and_order_correct']==False)]['advisor_name']

In [45]:
#ment[(ment['and_order_correct']=="not_include")]['advisor_name'

In [46]:
# def set_gap_wit_and(name):
#     if type(name)==str:
#         name=re.sub(r"(And)",r"\1;\2\3",name)
#     return name

In [47]:
ment.loc[ment['and_order_correct']==False,'advisor_name']=ment[ment['and_order_correct']==False]['advisor_name'].str.strip().str.replace(r'^And\s',"")

In [48]:
ment.loc[(ment['and_order_correct'].isin([True, False])),'advisor_name']=ment[(ment['and_order_correct'].isin([True, False]))]['advisor_name'].str.replace("And",";")

In [49]:
ment['advisor_name']=ment['advisor_name'].replace('%',';')

In [50]:
tmp = ment['advisor_name'].str.split(';').apply(pd.Series, 1).stack()
tmp.index = tmp.index.droplevel(-1)
tmp.name = 'advisor_name'
ment.drop(["advisor_name"],inplace=True, axis=1)
ment=ment.join(tmp)
ment.reset_index(drop=True, inplace=True)

In [51]:
ment[ment["previous_advisor_name"].str.contains(regex_pat1)][['previous_advisor_name',"advisor_name"]][250:300]

,previous_advisor_name,advisor_name
153960,"Rai, Gp Co-guide Jaiswal, Yk","Rai, Gp"
153961,"Rai, Gp Co-guide Jaiswal, Yk","Jaiswal, Yk"
153992,"Arora, Meera Co-guide Shukla, Veena","Arora, Meera"
153993,"Arora, Meera Co-guide Shukla, Veena","Shukla, Veena"
154287,"Guide: Rajini, P. S.","Rajini, P. S."
154288,"Muralidhara, Co-guide:",Muralidhara
155532,"Singh, Ritesh B. Patel / Ravindra Pal",Ritesh B. Patel
155533,"Singh, Ritesh B. Patel / Ravindra Pal",Ravindra Pal Singh
155546,"Amnekar, Ritu M. Gilhotra/ Nd",Ritu M. Gilhotra
155547,"Amnekar, Ritu M. Gilhotra/ Nd",Nd Amnekar


In [52]:
unwanted_char=['\)','\(',r'\d+']

In [53]:
# ment['advisor_name']=ment['advisor_name'].str.replace("\d+","")
# ment['researcher_name']=ment['researcher_name'].str.replace("\d+","")

In [54]:
# ment['advisor_name']=ment['advisor_name'].str.strip()
# ment['researcher_name']=ment['researcher_name'].str.strip()

In [55]:
ment['advisor_name']=ment['advisor_name'].replace(regex=unwanted_char,value="")
ment['researcher_name']=ment['researcher_name'].replace(regex=unwanted_char,value="")

In [56]:
ment['advisor_name']=ment['advisor_name'].str.strip(":- ")
ment['researcher_name']=ment['researcher_name'].str.strip(":- ")

In [57]:
ment['advisor_name']=ment['advisor_name'].str.lstrip(",")
ment['researcher_name']=ment['researcher_name'].str.lstrip(",")

In [58]:
# ment['advisor_name']=ment['advisor_name'].str.replace("\(","")
# ment['researcher_name']=ment['researcher_name'].str.replace("\(","")

In [59]:
ment[ment["previous_advisor_name"].str.contains("Ali Hasan Co Supervisor Khan, Ra")][['previous_advisor_name',"advisor_name"]]

,previous_advisor_name,advisor_name
81920,"Ali Hasan Co Supervisor Khan, Ra",Ali Hasan
81921,"Ali Hasan Co Supervisor Khan, Ra","Khan, Ra"


In [60]:
ment.loc[ment['advisor_name'].str.contains("M. Afzal :khan ,."),'advisor_name']="M. Afzal khan"
ment.loc[ment['advisor_name'].str.contains("Barve, S, S,"),'advisor_name']="Barve, S. S."
ment.loc[ment['advisor_name'].str.contains("Subramanin, V. Jain, V. K, Ramanathan, A. L."),'advisor_name']="Subramanin, V., Jain, V. K, Ramanathan, A. L."
ment.loc[ment['advisor_name'].str.contains("Bhatt, D, C,"),'advisor_name']="Bhatt, D. C."
ment.loc[ment['advisor_name'].str.contains("Reddy, C, R, K."),'advisor_name']="Reddy, C. R. K."
ment.loc[ment['advisor_name'].str.contains("Savani, Suresh, R,"),'advisor_name']="Savani, Suresh R."
ment.loc[ment['advisor_name'].str.contains("Vaghela, H, N,"),'advisor_name']="Vaghela, H. N."
ment.loc[ment['advisor_name'].str.contains("Mohd Aqill, Ahmad, Farhan Jalees, Mushir Ali"),'advisor_name']="Mohd Aqill; Ahmad, Farhan Jalees; Mushir Ali."

In [61]:
tmp = ment['advisor_name'].str.split(':').apply(pd.Series, 1).stack()
tmp.index = tmp.index.droplevel(-1)
tmp.name = 'advisor_name'
ment.drop(["advisor_name"],inplace=True, axis=1)
ment=ment.join(tmp)
ment.reset_index(drop=True, inplace=True)

In [62]:
def preprocessing_name(name):
    name= name.strip(",")
    names=name.split(',')
    names= [a for a in names if a.strip() != ""]
    if len(names)%2==1:
        names=names+[""]
    names = [','.join([i,j]) for i,j in zip(names[0::2], names[1::2])]
    names = ";".join(names)
    return names

In [63]:
ment.loc[(ment['advisor_name'].str.count(",")>2),'advisor_name']=ment[ment['advisor_name'].str.count(",")>2]['advisor_name'].apply(lambda x: preprocessing_name(x))

In [64]:
tmp = ment['advisor_name'].str.split(';').apply(pd.Series, 1).stack()
tmp.index = tmp.index.droplevel(-1)
tmp.name = 'advisor_name'
ment.drop(["advisor_name"],inplace=True, axis=1)
ment=ment.join(tmp)
ment.reset_index(drop=True, inplace=True)

In [65]:
ment['advisor_name']=ment['advisor_name'].str.strip()
ment['researcher_name']=ment['researcher_name'].str.strip()

In [66]:
#ment[ment['advisor_name'].str.contains(":")]#[['previous_advisor_name','advisor_name']]

In [67]:
ment.columns

Index(['advisorId', 'researcherId', 'researcher_name', 'date_submitted',
       'dc.language.iso[]', 'publisher_dept', 'N_DepartmentId_1',
       'publisher_institution', 'instituteId', 'title', 'thesisId',
       'dc.type.degree', 'dc.date.awarded', 'dc.subject.ddc',
       'dc.description.abstract', 'dc.title.alternative', 'N_DepartmentId',
       'uniq_ddc_across_dept', 'DepartmentId', 'previous_advisor_name',
       'and_order_correct', 'advisor_name'],
      dtype='object')

In [68]:
ment.shape

(205770, 22)

In [69]:
#ment.drop(['previous_advisor_name','and_order_correct','N_DepartmentId'],axis=1,inplace=True)

In [70]:
ment=ment[['advisorId', 'researcherId','advisor_name','researcher_name','dc.language.iso[]', 'publisher_dept',"DepartmentId",
       'publisher_institution', 'instituteId', 'title', 'thesisId','dc.type.degree', 'date_submitted','dc.date.awarded', 'dc.subject.ddc',
       'dc.description.abstract', 'dc.title.alternative','uniq_ddc_across_dept', 'N_DepartmentId_1','N_DepartmentId','previous_advisor_name']]

In [71]:
ment.to_csv(folder+"processed_sodhganga_mentorship_dept_rev3.csv", sep=",", index=False)

In [72]:
#ment2.loc[ment2['advisor_name'].str.count(",")>1,'advisor_name']=ment2.loc[ment2['advisor_name'].str.count(",")>1]['advisor_name'].str.replace(r"","")

In [73]:
#ment[ment['advisor_name']==","]['advisor_name']

In [74]:
#ment[ment['previous_advisor_name'].str.contains('And')][['previous_advisor_name',"advisor_name"]].shape

In [75]:
#ment[ment['advisor_name']=="Adhikari, Hiralal"]

In [76]:
ment.isna().sum() #null values count

advisorId                       0
researcherId                    0
advisor_name                    0
researcher_name                 0
dc.language.iso[]               0
publisher_dept                  0
DepartmentId                    0
publisher_institution           0
instituteId                     0
title                           0
thesisId                        0
dc.type.degree                  0
date_submitted              14723
dc.date.awarded            156171
dc.subject.ddc               3692
dc.description.abstract    188403
dc.title.alternative       200630
uniq_ddc_across_dept            0
N_DepartmentId_1                0
N_DepartmentId                  0
previous_advisor_name           0
dtype: int64

In [77]:
ment[ment["previous_advisor_name"].str.contains("Bhembre")][['previous_advisor_name','advisor_name']].values

array([['Bhembre, Uday À¤¡à¤¿à¤¸à¥\x8cà¤\x9dà¤¾, À¤\x9aà¤\x82à¤¦à¥\x8dà¤°à¤²à¥\x87à¤\x96à¤¾ À¤\x94à¤° À¤\xadà¥\x87à¤\x82à¤¬à¥\x8dà¤°à¥\x87 , À¤\x89à¤¦à¤¯',
        'Bhembre, Uday'],
       ['Bhembre, Uday', 'Bhembre, Uday'],
       ['Bhembre, Uday À¤¸à¤°à¤¦à¥\x87à¤¸à¤¾à¤¯, À¤®à¤¾à¤§à¤µà¥\x80 À¤\x94à¤° À¤\xadà¥\x87à¤\x82à¤¬à¥\x8dà¤°à¥\x87, À¤\x89à¤¦à¤¯',
        'Bhembre, Uday'],
       ['Bhembre, Uday', 'Bhembre, Uday'],
       ['Bhembre, Uday', 'Bhembre, Uday']], dtype=object)

In [78]:
#ment[ment["previous_advisor_name"].str.count(",")>2][['previous_advisor_name','advisor_name']].values[0:500]

In [79]:
#ment[ment['advisorId']==0]
ment[ment['previous_advisor_name'].str.contains('Vaishnavi, Samanthaand Singh, Shashank K.')]

,advisorId,researcherId,advisor_name,researcher_name,dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,title,...,dc.type.degree,date_submitted,dc.date.awarded,dc.subject.ddc,dc.description.abstract,dc.title.alternative,uniq_ddc_across_dept,N_DepartmentId_1,N_DepartmentId,previous_advisor_name
178598,65805,214058,"Vaishnavi, Samantha","Pandita, Archana",eng,school of biotechnology,D1417,Shri Mata Vaishno Devi University,I370,Study of synergistic effect of dietary molecul...,...,phd,2015-01-01,NaT,600::Technology|660::Chemical engineering,NaN,NaN,"('600', '660')",D1417,D1417,"Vaishnavi, Samanthaand Singh, Shashank K."
178599,65805,214058,"Singh, Shashank K.","Pandita, Archana",eng,school of biotechnology,D1417,Shri Mata Vaishno Devi University,I370,Study of synergistic effect of dietary molecul...,...,phd,2015-01-01,NaT,600::Technology|660::Chemical engineering,NaN,NaN,"('600', '660')",D1417,D1417,"Vaishnavi, Samanthaand Singh, Shashank K."


In [80]:
ment[ment['previous_advisor_name']=='Kand Rath, P. C.']#Bamezai, R N K and Rath, P C#['title'].values

,advisorId,researcherId,advisor_name,researcher_name,dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,title,...,dc.type.degree,date_submitted,dc.date.awarded,dc.subject.ddc,dc.description.abstract,dc.title.alternative,uniq_ddc_across_dept,N_DepartmentId_1,N_DepartmentId,previous_advisor_name
152550,56504,192703,"Bamezai, R N K","Arora, Reetakshi",eng,school of life sciences,D738,Jawaharlal Nehru University,I85,Study of T-cell response to hepatitis-E virus ...,...,phd,2003-01-01,NaT,500::Natural sciences & mathematics|570::Life ...,NaN,NaN,"('500', '570')",D738,D738,"Kand Rath, P. C."
152551,56504,192703,"Rath, P C","Arora, Reetakshi",eng,school of life sciences,D738,Jawaharlal Nehru University,I85,Study of T-cell response to hepatitis-E virus ...,...,phd,2003-01-01,NaT,500::Natural sciences & mathematics|570::Life ...,NaN,NaN,"('500', '570')",D738,D738,"Kand Rath, P. C."


In [81]:
# "Rakesh k. jat"
# "Rakesh jat"
# "Rakesh kumar jat"

In [82]:
ment[ment['researcherId']==192703]#['title'].values

,advisorId,researcherId,advisor_name,researcher_name,dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,title,...,dc.type.degree,date_submitted,dc.date.awarded,dc.subject.ddc,dc.description.abstract,dc.title.alternative,uniq_ddc_across_dept,N_DepartmentId_1,N_DepartmentId,previous_advisor_name
152550,56504,192703,"Bamezai, R N K","Arora, Reetakshi",eng,school of life sciences,D738,Jawaharlal Nehru University,I85,Study of T-cell response to hepatitis-E virus ...,...,phd,2003-01-01,NaT,500::Natural sciences & mathematics|570::Life ...,NaN,NaN,"('500', '570')",D738,D738,"Kand Rath, P. C."
152551,56504,192703,"Rath, P C","Arora, Reetakshi",eng,school of life sciences,D738,Jawaharlal Nehru University,I85,Study of T-cell response to hepatitis-E virus ...,...,phd,2003-01-01,NaT,500::Natural sciences & mathematics|570::Life ...,NaN,NaN,"('500', '570')",D738,D738,"Kand Rath, P. C."


In [83]:
#ment[ment['advisor_name'].isin(['Shastri, Paramanand','Shastri, Pramanand'])]['researcher_name'].values

In [84]:
#ment.loc[(ment['advisor_name'].str.count(",")>2),'advisor_name']=

In [85]:
# ['Singh, S, P.', 'Singh, S, P.'],
#        ['Singh, U, P.', 'Singh, U, P.'],

In [86]:
# ['P, C, Sekar', 'P, C, Sekar'],
#        ['P, C, Sekar', 'P, C, Sekar'],
#        ['P, C, Sekar', 'P, C, Sekar'],
#        ['P, C, Sekar', 'P, C, Sekar'],

In [ ]:
#different advId
Kosambi, D. D.
Kesambi, D. D.
Siddiqui, Athar H.
Siddiqi, Ather H.
Feza, Tabassum Azmi
Azmi, Feza Tavassaum
Ramlochan
Ranlochan
Bhullar, J.
Bhullar, Jatinder